<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/andrea_branch/federated_model_visual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing DINO and installing its dependencies

In [ ]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 11.86 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitl


# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split,DataLoader

In [ ]:
# transform_train = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                          std=(0.229, 0.224, 0.225))
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])





In [ ]:


full_train = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)





100%|██████████| 169M/169M [00:03<00:00, 43.1MB/s]


# Set Hyperparameters regarding the data spliting here!

In [ ]:
#@title set the parameters here!!

number_of_clients = None
train_frac = 0.8 #@param
val_frac = 0.2 #@param
is_seed_fixed = True #@param{type:"boolean"}
seed = 42 #@param{type:"integer"}




#@markdown </br> <h5>Indicate the number of clients that contribute in training:</h5>
n_clients = 10 #@param{type:"integer"}

#@markdown </br></br> <b>splitting hyperparameters</b>

spliting_method = "non-i.i.d. sharing" #@param["i.i.d. sharing","non-i.i.d. sharing"]




In [ ]:
#@title Set the parameters here only if <b>non-i.i.d. sharing</b> method had been selected!!
#@markdown Nc is the number of classes that each subset can contain
if spliting_method == "non-i.i.d. sharing":
  Nc = 10 #@param{type:"integer"}

  # are_classes_overlaping = False #@param{type:"boolean"}

#@markdown <h3>If we consider the Number of classes M and nummber of client K then:</h3>
#@markdown <ul>
#@markdown   <li>Nc should be:
#@markdown     <ul>
#@markdown       <li>
#@markdown         Greater than or equal to <b>\\(\frac{M}{K}\\)</b>
#@markdown       </li>
#@markdown       <li>
#@markdown         Less than or equal to K </b>
#@markdown       </li>
#@markdown     </ul>
#@markdown   </li>
#@markdown   <li>
#@markdown   Muximum number of clients means all classes contribute in every client
#@markdown   </li>

#@markdown </ul>


#@markdown </br></br><h3>Combination of classes are randomly selected which suits definition of federated learning especially Cross-device federated learning</h3>





# Data splitting

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np

generator = torch.Generator().manual_seed(seed)

total_size = len(full_train)
train_size = int(train_frac * total_size)
val_size   = total_size - train_size

train_set, val_set = random_split(full_train, [train_size, val_size], generator=generator)
train_indices = torch.tensor(train_set.indices)
val_indices = torch.tensor(val_set.indices)

train_set = Subset(train_set.dataset, train_indices)
val_set = Subset(val_set.dataset, val_set.indices)

train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False)
val_loader  =  DataLoader(val_set, batch_size=len(val_set), shuffle=False)



print(f"Train dataset size: {len(train_set)}")
print(f"Validation dataset size: {len(val_set)}")

lenghts = [train_size//n_clients] * n_clients

for i in range(train_size % n_clients):
  lenghts[i] += 1
print("Size of subset: ", lenghts)




# if spliting_method == "i.i.d. sharing":


Train dataset size: 40000
Validation dataset size: 10000
Size of subset:  [4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000]


In [ ]:
# @title split data dased on number of clients and with respect of label proportionality
import numpy as np
import random


def proportional_split(dataset, n_clients):

  full_train_indices = dataset.indices
  full_train_labels = torch.from_numpy(np.array(dataset .dataset.targets)[full_train_indices]) #collects labels from all dataset
  unique_lables = torch.unique(full_train_labels) #Removes dupilication and generates a uniuqe list of labels (classes)
  proportionality ={}
  classes_indices = {}


  for i in unique_lables:
    proportionality[i] =( full_train_labels == i).sum() / len(full_train_labels) #Calculates proportinality of each class
    classes_indices[i] = torch.nonzero(full_train_labels == i).squeeze() #Collects and save Indices in an array based on classes

  for i in classes_indices.keys():
    classes_indices[i] = classes_indices[i][torch.randperm(classes_indices[i].shape[0])] #suffels the indices

  client_data_size = len(full_train_labels) // n_clients #Minimum dataset size of each client
  client_indices = {}

  #For each client we generate a element in client_indices dict to keep track of indices we'll associated with each client
  for client in range(n_clients):
    if not client_indices.get(client):
      client_indices[client] = torch.empty(0, dtype=torch.long)
  #__________________


  #For each client we calculate how many samples from each specific label should be seperated. We take out the requried number of them form the list
    for label in proportionality.keys():
      pointer = int(proportionality[label] * client_data_size )
      pointer = pointer if pointer < classes_indices[label].size()[0] else classes_indices[label].size()[0]
      client_indices[client] = torch.cat((client_indices[client], classes_indices[label][:pointer]), dim=0)
      classes_indices[label] = classes_indices[label][pointer:]


  #After spliting data we distribute remaining samples amoung the clients
  for label in classes_indices.keys():
    while True:
      for client in client_indices.keys():

        if classes_indices[label].size()[0] == 0:
          break
        client_indices[client] = torch.cat((
            client_indices[client],
            classes_indices[label][:1] ),
            dim=0)


        classes_indices[label] = classes_indices[label][1:]

      if classes_indices[label].size()[0] == 0:
        break


  #spilt actual dataset to mulitiple subset for clients
  client_data={
      client_id: Subset(dataset.dataset,indices[torch.randperm(len(indices))])
      for client_id, indices in client_indices.items()
  }
  return client_data



client_data = proportional_split(train_set, n_clients)
for client_id in client_data.keys():
  print(f"Client {client_id} has {len(client_data[client_id])} samples")





Client 0 has 4040 samples
Client 1 has 4031 samples
Client 2 has 4023 samples
Client 3 has 4018 samples
Client 4 has 4009 samples
Client 5 has 3999 samples
Client 6 has 3988 samples
Client 7 has 3975 samples
Client 8 has 3965 samples
Client 9 has 3952 samples


# Log System

In this section Requerd Data will be archaved.<br/><br/>
**Archaving this inforamtion will make it possible to:**
*   Handel Clients
*   Manage the models
*   Keep track of results of different Backbones
*   Compare measurement criteria
*   Handel model merging process
*   Save path to the models

<br/><br/>
**These data will be saved in two seperted csv file to :**

1.   Archave the LOCAL Models  
2.   Archave the GLOBAL Models resulted by each round

<br/><br/>
The csv files will be handeled as panda.dataframe and each row in the csv file addresses one of models
<br/>

**Columns (COMMON):**<br/>
1. Backbone model name
2. Model name
3. Path
4. Time of log
5. Measurement criteria
 * loss
 * Accuracy
 * ...?
6. Size of dataset

**Columns (Local Models only):**<br/>
7. Client Id
8. Classes (Indicate which classes have been covered by each client)(format:"2,4,63,80,9" or "all" for all the classes)
9. Round number
10. Duration of training
11. Train Test ratio

**Columns (Global Models only):**<br/>
7. Number of clients
7. Number of rounds
8. Model Aggregation method







In [ ]:
# @title Functions
import torch
from datetime import datetime
import time
from google.colab import drive


def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time



tic_start_time = None

def tic():
    global tic_start_time
    tic_start_time = time.perf_counter() # start the timer

def toc():
    if tic_start_time is None:
        print("Error: You must call tic() before toc()")
        return None
    elapsed_time = time.perf_counter() - tic_start_time
    return elapsed_time


def load_storage():
  drive.mount('/content/drive')
  path_to_clients = '/content/drive/MyDrive/MLDL_FederatedLearning/models/clients/'
  path_to_global  = '/content/drive/MyDrive/MLDL_FederatedLearning/models/global/'
  return path_to_clients, path_to_global




def save_client(model,client,round_number,backbone,path_to_clients):
  round_number = str(round_number)
  round_number = "0"*(4-len(round_number)) + round_number

  client = str(client)
  client = "0"*(4-len(client)) + client

  model_name = backbone + "_" + client + "_" + round_number + "pth"

  path =  path_to_clients + model_name

  numbertorch.save(model.state_dict(),  path )

  return model_name, path




def save_global_model(model,number_of_clients,number_of_round,backbone,path_to_global):
  number_of_round = str(number_of_round)
  number_of_round = "0"*(4-len(number_of_round)) + number_of_round

  number_of_clients = str(number_of_clients)
  number_of_clients = "0"*(4-len(number_of_clients)) + number_of_clients

  model_name = backbone + "_" + number_of_clients + "_" + number_of_round + "pth"

  path =  path_to_global + model_name

  numbertorch.save(model.state_dict(), path  )

  return model_name, path


In [ ]:
# Next we should work on merging